## BERTopic Model

### Import data

In [1]:
import os
import json
from gensim.test.utils import datapath

tmp = datapath(os.getcwd() + '\\tmp\\model.bertopicmodel')
dirpath = os.getcwd() + '\\raw_texts.json'

with open(dirpath, 'r') as f:
    documents = json.load(f)
    texts = [' '.join(doc) for doc in documents]

# List of stopwords
with open('swl_optimum.txt', 'r', encoding='utf-8') as f:
    swl = f.read().split('\n')

In [2]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(texts, columns=['headline_text'])
df.head()

,headline_text
0,9 март 2016 год произойти полный солнечный зат...
1,28 сентябрь состояться полный лунный затмение ...
2,20 март 2015 год произойти полный солнечный за...
3,22 июль 2009 год произойти самый продолжительн...
4,первый упоминание о солнечный затмение относит...


### Create a custom embedding model

In [3]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

### Dimensionality reduction & clustering

In [4]:
from umap import UMAP
from hdbscan import HDBSCAN

umap_model = UMAP(n_neighbors=20, n_components=1, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=15, min_samples=3, \
                        gen_min_span_tree=True, prediction_data=True)

### Create a custom vectorizer model

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

# Filter out tokens by frequency
vectorizer_model = CountVectorizer(stop_words=swl, ngram_range=(1,2), min_df=10, max_df=0.9) 

### Initialize and run the BERTopic model

In [6]:
from bertopic import BERTopic

bertopic_model = BERTopic(top_n_words=15, language='multilingual', \
                          embedding_model=embedding_model, umap_model=umap_model, \
                          hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, \
                          calculate_probabilities=True, verbose=True)

topics, probs = bertopic_model.fit_transform(df.headline_text)

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

2022-06-19 01:47:54,519 - BERTopic - Transformed documents to Embeddings
2022-06-19 01:48:02,213 - BERTopic - Reduced dimensionality
2022-06-19 01:48:02,352 - BERTopic - Clustered reduced embeddings


In [8]:
# Save the model
bertopic_model.save(tmp)

In [9]:
# Load the model
BERTopic.load(tmp)

### Reduce the number of topics (optional)

In [7]:
new_topics, new_probs = bertopic_model.reduce_topics(df.headline_text, topics, probs, \
                                                     nr_topics=15)

2022-06-19 01:48:17,758 - BERTopic - Reduced number of topics from 37 to 16


### Print the results

In [10]:
# -1 refers to all outliers and should typically be ignored
freq = bertopic_model.get_topic_info()
freq.head(10)

,Topic,Count,Name
0,0,130,0_частица_физика_распад_электрон
1,-1,117,-1_минута_космический_солнце_вселенная
2,1,117,1_ген_геном_мутация_генетический
3,2,109,2_магнитный_атом_электрон_ток
4,3,99,3_белок_нейрон_фермент_мозг
5,4,92,4_излучение_космический_вселенная_солнце
6,5,89,5_кость_лес_популяция_каменный
7,6,78,6_личинка_червь_ген_многоклеточный
8,7,68,7_рак_мутация_ген_ткань
9,8,66,8_рыба_палеонтолог_геном_млекопитающее


In [11]:
bertopic_model.get_topics()

{-1: [('минута', 0.03804057552550629),
  ('космический', 0.03343264724792471),
  ('солнце', 0.032242861361881196),
  ('вселенная', 0.025057397706515803),
  ('земной', 0.022592425841297524),
  ('частица', 0.02059300715520887),
  ('диск', 0.02038276541328327),
  ('излучение', 0.019914212099201318),
  ('секунда', 0.019812774982457263),
  ('магнитный', 0.019409288071367846),
  ('вирус', 0.01892137306660891),
  ('электрон', 0.018195098674246006),
  ('атмосфера', 0.01790017481484392),
  ('градус', 0.016024211054446038),
  ('порода', 0.015835569676347437)],
 0: [('частица', 0.07602925169179675),
  ('физика', 0.07068909272060787),
  ('распад', 0.036064994580356315),
  ('электрон', 0.03424019496551965),
  ('фотон', 0.029602718266056854),
  ('атом', 0.027874776996832013),
  ('теоретик', 0.024707266224254287),
  ('отклонение', 0.023093435960516585),
  ('материя', 0.022672636095237945),
  ('элементарный', 0.022338340334804283),
  ('излучение', 0.021017822703882912),
  ('электронный', 0.02062658982

In [12]:
# Select the most frequent topic
bertopic_model.get_topic(0)

[('частица', 0.07602925169179675),
 ('физика', 0.07068909272060787),
 ('распад', 0.036064994580356315),
 ('электрон', 0.03424019496551965),
 ('фотон', 0.029602718266056854),
 ('атом', 0.027874776996832013),
 ('теоретик', 0.024707266224254287),
 ('отклонение', 0.023093435960516585),
 ('материя', 0.022672636095237945),
 ('элементарный', 0.022338340334804283),
 ('излучение', 0.021017822703882912),
 ('электронный', 0.020626589828400092),
 ('рождение', 0.020254674968186125),
 ('столкновение', 0.018632069272162703),
 ('симметрия', 0.01760962849519374)]